####  Assignment 11

**Sai Gauthami Kuravi**

<ol>
    <li>Construct, train and test a neural network to predict high utilization in the next one year using recurrent neural networks. </li>
    <li>You should use tensorflow library to do so</li>
    <li>Please submit python notebook and html version of the output showing results. </li>

</ol>


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.metrics import accuracy_score,roc_auc_score

In [2]:
import os
os.getcwd()
os.chdir(r'C:\Users\skura\Desktop')
df = pd.read_csv('exlix_binary_claims_w_dates.csv')


In [3]:
back_window = 20 # this is number of claims
shift = 1 # this is number of claims
# forward window defined in terms of days
forward_window = 365

In [4]:
#
# split in training/testing by patient
#
from sklearn.model_selection import train_test_split

pts = df['patient_id'].unique()

trp, tsp = train_test_split(pts)

In [5]:
tr = df[df['patient_id'].isin(trp)]
ts = df[df['patient_id'].isin(tsp)]

In [6]:
x_tr = [] # inputs
y_tr = [] # outputs
for p in trp[:10000]:
    pt = tr[tr['patient_id'] == p]
    for t in range(back_window, len(pt)):
        # input variables
        x_tr.append(pt[t-back_window:t].iloc[:,2:])
    
        # output variable
        t_days = pt.iloc[t].claim_days_cum
        y_tr.append(len(pt[(pt['claim_days_cum'] > t_days) & (pt['claim_days_cum'] <= t_days + 365) ]))
    
        #stop looping if past year 2
        if pt.iloc[t].claim_days_cum > 730:
            break
        

In [7]:
x_ts = [] # inputs
y_ts = [] # outputs
for p in tsp[:1000]:
    pt = ts[ts['patient_id'] == p]
    for t in range(back_window, len(pt)):
        # input variables
        x_ts.append(pt[t-back_window:t].iloc[:,2:])
    
        # output variable
        t_days = pt.iloc[t].claim_days_cum
        y_ts.append(len(pt[(pt['claim_days_cum'] > t_days) & (pt['claim_days_cum'] <= t_days + 365) ]))
    
        #stop looping if past year 2
        if pt.iloc[t].claim_days_cum > 730:
            break

In [8]:
y_ts = (np.array(y_ts) >= 100).astype('int')
y_ts

array([0, 0, 0, ..., 1, 1, 1])

In [9]:
x_tr_vals = np.array([xx.values for xx in x_tr]).astype('float32')
x_ts_vals = np.array([xx.values for xx in x_ts]).astype('float32')

In [10]:
y_tr = (np.array(y_tr) >= 100).astype('int')

In [11]:
import tensorflow as tf

In [12]:
elix_lstm_model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(29, input_shape=(20,29)),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

elix_lstm_model.compile(optimizer='adam', loss='binary_crossentropy')

In [13]:
elix_lstm_model.fit(x_tr_vals, y_tr, epochs=40,
                      steps_per_epoch=None)

Train on 320794 samples
Epoch 1/40
320794/320794 [==============================] - 75s 234us/sample - loss: 0.4536
Epoch 2/40
320794/320794 [==============================] - 74s 229us/sample - loss: 0.4260
Epoch 3/40
320794/320794 [==============================] - 73s 229us/sample - loss: 0.4086
Epoch 4/40
320794/320794 [==============================] - 73s 227us/sample - loss: 0.3927
Epoch 5/40
320794/320794 [==============================] - 6711s 21ms/sample - loss: 0.3802
Epoch 6/40
320794/320794 [==============================] - 61s 190us/sample - loss: 0.3694
Epoch 7/40
320794/320794 [==============================] - 65s 204us/sample - loss: 0.3609
Epoch 8/40
320794/320794 [==============================] - 71s 222us/sample - loss: 0.3528
Epoch 9/40
320794/320794 [==============================] - 75s 235us/sample - loss: 0.3459
Epoch 10/40
320794/320794 [==============================] - 84s 260us/sample - loss: 0.3399
Epoch 11/40
320794/320794 [===========================

In [14]:
x_ts_vals.shape

(31645, 20, 29)

In [15]:
x_tr_vals.shape

(320794, 20, 29)

In [16]:
probs = elix_lstm_model.predict_proba(x_ts_vals)

In [17]:
probs

array([[0.06931208],
       [0.42002192],
       [0.3135628 ],
       ...,
       [0.01647905],
       [0.00721705],
       [0.00762643]], dtype=float32)

In [18]:
results = elix_lstm_model.evaluate(x_ts_vals, y_ts, batch_size=128)

31645/31645 [==============================] - 2s 48us/sample - loss: 0.6861


In [19]:
results

0.6860758767155867